# <center> Assignment 2 - Computer Vision ENCS5343


### This assignment achieve to implement Content-Based Image Retrieval (CBIR) using various image representations in order to compare between them, observe the results of each method, measure the efficiency of each method using metrics (Accuracy, AUC, F1-Score, ....)
#### Steps: 
1. Data processing (uploading images and getting them ready).
2. Choose an image representation technique.
3. Apply the CBIR on 15 different queries.
4. Observe the results depending with their rankings.
5. Measure the preformace using different measure metrics.

### Step 0: Import important packages and libraries.

In [5]:
from PIL import Image
import cv2 as cv
import matplotlib.pyplot
import os

### Step 1: Upload and store the data.

In [6]:
def upload_and_store_images(dataset_filepath):
    images = []
    for filename in os.listdir(path=dataset_filepath):
        image_path = os.path.join(dataset_filepath, filename)
        if os.path.isfile(image_path):
            try: 
                images.append(Image.open(image_path))
            except:
                print("Failed to upload image: {image_path}")
    return images
images = upload_and_store_images('./dataset')